## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_1 = pd.read_csv("../../바이너리_최종데이터_1004/gcc6/o1/o1_bincore6.csv", index_col=0)
print(bin6_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_1.reset_index(inplace=True, drop=True)

print('shape -', bin6_1.shape)
print('reset_index 완료')
print('input data shape')
bin6_1.head()

(27602871, 2)
shape - (27602871, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_1['label'].value_counts())

256
0    27564156
1       38715
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_1[bin6_1['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 16 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_1), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin6_1', len(ls_idx_bin))

# loc 로 수정필요
bin6_1_Ngram = bin6_1.loc[ls_idx_bin,:].copy()

619440
나머지 0
최종 길이 619440
bin6_1 619440


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_1_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_1.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin6_1[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

38715.0
0 14055531
1000 27109914
2000 25004165
3000 25874522
4000 25403398
5000 14915842
6000 9082704
7000 26711874
8000 23401947
9000 2749731
10000 19370757
11000 18541485
12000 15321785
13000 4545860
14000 11901860
15000 9866727
16000 2259722
17000 16944194
18000 22788516
19000 17462754
20000 22330276
21000 4751689
22000 3283346
23000 24347385
24000 19231701
25000 10553097
26000 24343217
27000 2445851
28000 12540906
29000 5798943
30000 26043278
31000 6328282
32000 7307307
33000 2205008
34000 3335291
35000 20213995
36000 8481643
37000 27209385
38000 2588500
완료
38715


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin6_1['label'].value_counts()[1])

38715
38715


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin6_1_Ngram])
final.shape

(1238880, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_1_onehot_Ngram = pd.concat([final['label'], bc6_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_1_onehot_Ngram.shape)

원핫인코딩완료
(1238880, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc6_1 = bc6_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_1 = bc6_1_onehot_Ngram['label'].to_numpy()
print(x_bc6_1.shape, x_bc6_1.shape)

x_bc6_1 = x_bc6_1.reshape(-1, right_idx, x_bc6_1.shape[1])
y_bc6_1 = y_bc6_1.reshape(-1, right_idx, 1)

print(x_bc6_1.shape, y_bc6_1.shape)

(1238880, 256) (1238880, 256)
(77430, 16, 256) (77430, 16, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_1.shape[0])

x_bc6_1 = x_bc6_1[p]
y_bc6_1 = y_bc6_1[p]

print(x_bc6_1.shape, y_bc6_1.shape)

(77430, 16, 256) (77430, 16, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 10) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(24, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습

In [11]:
model1 = models.Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

print('======Training stage======')
model1.fit(x_bc6_1,
           y_bc6_1,
           epochs = 500,
           batch_size = 32,
           validation_split=0.3,
           callbacks=[early_stopping])

model1.save('gcc6_bin_core_s16_h24_o1.h5')
print('save 완료')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 16, 48)            53952     
_________________________________________________________________
time_distributed (TimeDistri (None, 16, 1)             49        
Total params: 54,001
Trainable params: 54,001
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 54201 samples, validate on 23229 samples
Epoch 1/500
54201/54201 [==============================] - 61s 1ms/sample - loss: 0.0363 - accuracy: 0.9907 - val_loss: 0.0100 - val_accuracy: 0.9971
Epoch 2/500
54201/54201 [==============================] - 49s 900us/sample - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.0048 - val_accuracy: 0.9986
Epoch 3/

save 완료
